In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

In [ ]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))
print(torch.backends.cudnn.version())

In [ ]:
import os   # 운영체제와의 상호작용을 위한 os 모듈 임포트
import json   # JSON 파일을 처리하기 위한 json 모듈 임포트
import torch   # PyTorch 프레임워크 임포트
import torchvision   # PyTorch의 컴퓨터 비전 패키지인 torchvision 임포트
from tqdm import tqdm   # 코드 실행 진행 상황을 시각화하기 위한 tqdm 모듈 임포트
from torchvision import transforms   # 이미지 변환을 위한 torchvision의 transforms 모듈 임포트
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor   # Faster R-CNN 모델의 커스터마이징을 위한 FastRCNNPredictor 임포트
from torch.utils.data import Dataset, DataLoader   # 데이터셋 및 데이터 로더 클래스 임포트
from PIL import Image   # 이미지 처리를 위한 Python Imaging Library(PIL)의 Image 클래스 임포트

# 사용자 정의 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, root_dir, transforms=None):   # 데이터셋 초기화 함수 정의
        self.root_dir = root_dir   # 이미지 및 레이블 파일이 저장된 루트 디렉토리 설정
        self.transforms = transforms   # 이미지 변환 설정
        self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "images"))))   # 이미지 파일 목록 불러오기 및 정렬
        self.labels = list(sorted(os.listdir(os.path.join(root_dir, "labels"))))   # 레이블 파일 목록 불러오기 및 정렬
        self.class_to_idx = {   # 클래스 이름을 인덱스로 매핑하는 딕셔너리 정의
            'car': 1, 'bus': 2, 'truck': 3, 'special vehicle': 4, 
            'motorcycle': 5, 'bicycle': 6, 'personal mobility': 7, 
            'person': 8, 'Traffic_light': 9, 'Traffic_sign': 10
        }

    def __len__(self):   # 데이터셋의 전체 크기를 반환하는 함수 정의
        return len(self.imgs)   # 이미지 파일의 개수를 반환

    def __getitem__(self, idx):   # 인덱스에 해당하는 데이터 반환 함수 정의
        img_path = os.path.join(self.root_dir, "images", self.imgs[idx])   # 인덱스에 해당하는 이미지 파일 경로 설정
        label_path = os.path.join(self.root_dir, "labels", self.labels[idx])   # 인덱스에 해당하는 레이블 파일 경로 설정

        img = Image.open(img_path).convert("RGB")   # 이미지를 RGB 모드로 열기
        original_width, original_height = img.size   # 원본 이미지의 가로, 세로 크기 가져오기

        # 이미지 크기를 512x512로 리사이즈
        new_width, new_height = 512, 512
        img = img.resize((new_width, new_height))   # 이미지 크기 변경

        with open(label_path) as f:   # 레이블 파일 열기
            annotation = json.load(f)   # JSON 형식의 레이블 파일 읽기

        boxes = []   # 바운딩 박스 좌표를 저장할 리스트 초기화
        labels = []   # 레이블을 저장할 리스트 초기화
        for obj in annotation['Annotation']:   # 각 객체에 대해 반복
            xmin, ymin, xmax, ymax = obj['data']   # 바운딩 박스 좌표 가져오기
            
            # 바운딩 박스 좌표를 리사이즈된 이미지에 맞게 조정
            xmin = xmin * (new_width / original_width)
            xmax = xmax * (new_width / original_width)
            ymin = ymin * (new_height / original_height)
            ymax = ymax * (new_height / original_height)
            
            boxes.append([xmin, ymin, xmax, ymax])   # 조정된 바운딩 박스 좌표 리스트에 추가
            labels.append(self.class_to_idx[obj['class_name']])   # 클래스 이름을 인덱스로 변환하여 레이블 리스트에 추가

        boxes = torch.as_tensor(boxes, dtype=torch.float32)   # 바운딩 박스 좌표를 텐서로 변환
        labels = torch.as_tensor(labels, dtype=torch.int64)   # 레이블을 텐서로 변환

        image_id = torch.tensor([idx])   # 이미지 ID를 텐서로 설정
        target = {}   # 타깃 딕셔너리 초기화
        target["boxes"] = boxes   # 바운딩 박스 좌표를 타깃 딕셔너리에 추가
        target["labels"] = labels   # 레이블을 타깃 딕셔너리에 추가
        target["image_id"] = image_id   # 이미지 ID를 타깃 딕셔너리에 추가

        if self.transforms is not None:   # 이미지 변환이 설정되어 있다면
            img = self.transforms(img)   # 이미지를 변환

        return img, target   # 이미지와 타깃 딕셔너리를 반환

# 이미지 변환을 정의하는 함수
def get_transform(train):
    transforms_list = []   # 변환 리스트 초기화
    transforms_list.append(transforms.ToTensor())  # 이미지를 텐서로 변환하는 변환 추가
    return transforms.Compose(transforms_list)   # 변환 리스트를 하나의 변환으로 결합하여 반환

# Pre-trained Faster R-CNN 모델을 사용하고 클래스 수에 맞게 수정하는 함수
def get_model(num_classes):
    # COCO 데이터셋으로 사전 학습된 모델 불러오기
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # 분류기(classifier)를 위한 입력 피처 수 가져오기
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # 사전 학습된 분류기를 새로운 분류기로 대체
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model   # 수정된 모델 반환

# 학습 및 평가 함수 정의
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()   # 모델을 학습 모드로 설정
    for images, targets in tqdm(data_loader, desc=f"Epoch {epoch+1}/10"):   # 에폭에 대해 반복하며 진행 상황을 표시
        images = list(image.to(device) for image in images)   # 이미지를 GPU로 이동
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]   # 타깃 데이터를 GPU로 이동

        loss_dict = model(images, targets)   # 모델에서 손실값 계산
        losses = sum(loss for loss in loss_dict.values())   # 모든 손실값의 합 계산

        optimizer.zero_grad()   # 옵티마이저의 그래디언트를 초기화
        losses.backward()   # 손실값을 통해 그래디언트 계산
        optimizer.step()   # 옵티마이저로 모델 파라미터 업데이트

    print(f"Epoch #{epoch}: Loss {losses.item()}")   # 에폭 종료 후 손실값 출력

# 메인 함수로 모델 학습 실행
def main():
    # 데이터 로딩 코드
    train_dataset = CustomDataset('training', get_transform(train=True))   # 학습 데이터셋 로드
    val_dataset = CustomDataset('validation', get_transform(train=False))   # 검증 데이터셋 로드
    
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=0, collate_fn=lambda x: tuple(zip(*x)))   # 학습 데이터 로더 생성
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=0, collate_fn=lambda x: tuple(zip(*x)))   # 검증 데이터 로더 생성

    # 학습을 CPU 대신 CUDA에서 실행
    device = torch.device('cuda')   # CUDA 디바이스 설정
    print(device)   # 디바이스 출력

    # 모델 설정
    num_classes = 11  # 10개의 클래스 + 배경 클래스 (background)
    model = get_model(num_classes)   # 모델 생성 및 클래스 수에 맞게 수정
    model.to(device)   # 모델을 CUDA 디바이스로 이동

    # 옵티마이저 설정
    params = [p for p in model.parameters() if p.requires_grad]   # 학습이 필요한 파라미터만 선택
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)   # SGD 옵티마이저 설정

    num_epochs = 10   # 학습할 에폭 수 설정
    for epoch in range(num_epochs):   # 각 에폭에 대해 반복
        train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=10)   # 한 에폭 동안 모델 학습
        torch.save(model.state_dict(), f"model_{epoch}.pth")   # 에폭마다 모델 가중치 저장

if __name__ == "__main__":   # 메인 프로그램 시작점
    main()   # 메인


In [ ]:
# faster rcnn 학습 코드 (2) - 모델 학습 중 실행을 중단했을 경우 저장된 모델 가중치를 불러와 중간부터 다시 학습

import os
import json
import torch
import torchvision
from tqdm import tqdm
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# CustomDataset 클래스 정의 - 데이터셋을 구성하는 클래스
class CustomDataset(Dataset):
    def __init__(self, root_dir, transforms=None):
        self.root_dir = root_dir  # 이미지와 라벨이 포함된 루트 디렉토리
        self.transforms = transforms  # 이미지에 적용할 변환들
        self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "images"))))  # 이미지 파일 리스트
        self.labels = list(sorted(os.listdir(os.path.join(root_dir, "labels"))))  # 라벨 파일 리스트
        # 클래스 이름을 인덱스로 매핑
        self.class_to_idx = {
            'car': 1, 'bus': 2, 'truck': 3, 'special vehicle': 4, 
            'motorcycle': 5, 'bicycle': 6, 'personal mobility': 7, 
            'person': 8, 'Traffic_light': 9, 'Traffic_sign': 10
        }

    def __len__(self):
        return len(self.imgs)  # 데이터셋의 길이 반환

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, "images", self.imgs[idx])  # 이미지 경로
        label_path = os.path.join(self.root_dir, "labels", self.labels[idx])  # 라벨 경로

        img = Image.open(img_path).convert("RGB")  # 이미지 열기 및 RGB로 변환
        original_width, original_height = img.size  # 원본 이미지 크기

        # GPU 메모리 부족 현상을 피하기 위해 이미지를 512x512로 리사이즈
        new_width, new_height = 512, 512
        img = img.resize((new_width, new_height))

        # 라벨 파일을 열고 JSON으로 파싱
        with open(label_path) as f:
            annotation = json.load(f)

        # 바운딩 박스와 라벨을 저장할 리스트 초기화
        boxes = []
        labels = []
        for obj in annotation['Annotation']:
            xmin, ymin, xmax, ymax = obj['data']
            
            # 바운딩 박스 좌표를 이미지 크기에 맞춰 조정
            xmin = xmin * (new_width / original_width)
            xmax = xmax * (new_width / original_width)
            ymin = ymin * (new_height / original_height)
            ymax = ymax * (new_height / original_height)
            
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(self.class_to_idx[obj['class_name']])

        # 바운딩 박스와 라벨을 텐서로 변환
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        # 이미지 ID 생성
        image_id = torch.tensor([idx])
        target = {}
        target["boxes"] = boxes  # 바운딩 박스 정보
        target["labels"] = labels  # 클래스 라벨 정보
        target["image_id"] = image_id  # 이미지 ID

        # 이미지 변환이 설정된 경우 변환 적용
        if self.transforms is not None:
            img = self.transforms(img)

        return img, target  # 이미지와 타겟 반환

# 데이터 변환 정의 함수
def get_transform(train):
    transforms_list = []
    transforms_list.append(transforms.ToTensor())  # 이미지를 텐서로 변환
    return transforms.Compose(transforms_list)

# Faster R-CNN 모델 정의 및 수정 함수
def get_model(num_classes):
    # COCO 데이터셋으로 사전 학습된 Faster R-CNN 모델 로드
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # 분류기 입력 피처의 수 가져오기
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # 사전 학습된 분류기를 새로운 분류기로 교체
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

# 한 에폭 동안의 학습 함수
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()  # 모델을 학습 모드로 설정
    for images, targets in tqdm(data_loader, desc=f"Epoch {epoch+1}/10"):  # 데이터 로더에서 배치 단위로 데이터 가져오기
        images = list(image.to(device) for image in images)  # 이미지를 GPU로 이동
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]  # 타겟도 GPU로 이동

        loss_dict = model(images, targets)  # 모델에 이미지와 타겟 입력하고 손실 계산
        losses = sum(loss for loss in loss_dict.values())  # 모든 손실 값 합산

        optimizer.zero_grad()  # 옵티마이저의 기울기 초기화
        losses.backward()  # 역전파 수행
        optimizer.step()  # 옵티마이저 업데이트

    print(f"Epoch #{epoch}: Loss {losses.item()}")  # 에폭의 손실 출력

# 모델 학습을 위한 메인 함수
def main():
    # 데이터셋 로딩
    train_dataset = CustomDataset('training', get_transform(train=True))  # 학습 데이터셋
    val_dataset = CustomDataset('validation', get_transform(train=False))  # 검증 데이터셋
    
    # 데이터 로더 생성
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=0, collate_fn=lambda x: tuple(zip(*x)))
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=0, collate_fn=lambda x: tuple(zip(*x)))

    # 디바이스 설정 (GPU 사용)
    device = torch.device('cuda')
    print(device)

    # 모델 생성
    num_classes = 11  # 10개의 클래스 + 배경 클래스
    model = get_model(num_classes)
    model.to(device)  # 모델을 GPU로 이동

    # 저장된 모델 상태 불러오기
    model_path = 'model_0.pth'
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))  # 모델의 상태 불러오기
        print(f"Loaded model weights from {model_path}")
    else:
        print(f"Model file {model_path} not found. Starting training from scratch.")

    # 옵티마이저 설정
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

    num_epochs = 10  # 학습할 에폭 수
    for epoch in range(1, num_epochs):
        train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=10)  # 한 에폭 학습

        # 모델 상태 저장
        torch.save(model.state_dict(), f"model_{epoch}.pth")
        
        # 옵티마이저 상태 저장
        torch.save(optimizer.state_dict(), f"optimizer_{epoch}.pth")

if __name__ == "__main__":
    main()  # 메인 함수 실행

In [ ]:
import os   # 운영체제와의 상호작용을 위한 os 모듈 임포트
import json   # JSON 파일을 처리하기 위한 json 모듈 임포트
import torch   # PyTorch 프레임워크 임포트
import torchvision   # PyTorch의 컴퓨터 비전 패키지인 torchvision 임포트
from torchvision import transforms   # 이미지 변환을 위한 torchvision의 transforms 모듈 임포트
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor   # Faster R-CNN 모델의 커스터마이징을 위한 FastRCNNPredictor 임포트
from torch.utils.data import Dataset, DataLoader   # 데이터셋 및 데이터 로더 클래스 임포트
from PIL import Image   # 이미지 처리를 위한 Python Imaging Library(PIL)의 Image 클래스 임포트
from tqdm import tqdm   # 코드 실행 진행 상황을 시각화하기 위한 tqdm 모듈 임포트

# 사용자 정의 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, root_dir, transforms=None):   # 데이터셋 초기화 함수 정의
        self.root_dir = root_dir   # 이미지 파일이 저장된 루트 디렉토리 설정
        self.transforms = transforms   # 이미지 변환 설정
        self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "images"))))   # 이미지 파일 목록 불러오기 및 정렬

    def __len__(self):   # 데이터셋의 전체 크기를 반환하는 함수 정의
        return len(self.imgs)   # 이미지 파일의 개수를 반환

    def __getitem__(self, idx):   # 인덱스에 해당하는 데이터 반환 함수 정의
        img_path = os.path.join(self.root_dir, "images", self.imgs[idx])   # 인덱스에 해당하는 이미지 파일 경로 설정
        img = Image.open(img_path).convert("RGB")   # 이미지를 RGB 모드로 열기

        if self.transforms is not None:   # 이미지 변환이 설정되어 있다면
            img = self.transforms(img)   # 이미지를 변환

        image_id = os.path.splitext(self.imgs[idx])[0]  # 이미지 파일 이름에서 확장자를 제거하고 ID 추출
        return img, image_id   # 이미지와 이미지 ID를 반환

# 이미지 변환을 정의하는 함수
def get_transform():
    transforms_list = []   # 변환 리스트 초기화
    transforms_list.append(transforms.ToTensor())   # 이미지를 텐서로 변환하는 변환 추가
    return transforms.Compose(transforms_list)   # 변환 리스트를 하나의 변환으로 결합하여 반환

# 사전 학습된 모델을 사용하고 저장된 상태를 불러오는 함수
def get_model(num_classes, model_path):
    # COCO 데이터셋으로 사전 학습된 모델을 불러오고 분류기 헤드를 교체
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features   # 분류기(classifier)를 위한 입력 피처 수 가져오기
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)   # 사전 학습된 분류기를 새로운 분류기로 대체
    
    # 학습된 모델 가중치를 불러오기
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda')))   # 모델 가중치 로드
    return model   # 모델 반환

# 예측 결과를 COCO 형식으로 저장하는 함수
def save_predictions_as_coco_format(model, data_loader, device, output_file="predictions.json"):
    model.eval()   # 모델을 평가 모드로 설정
    results = []   # 예측 결과를 저장할 리스트 초기화

    with torch.no_grad():   # 그래디언트를 계산하지 않도록 설정
        for images, image_ids in tqdm(data_loader, desc="Generating Predictions"):   # 데이터 로더에서 배치를 반복하며 진행 상황을 표시
            images = list(img.to(device) for img in images)   # 이미지를 GPU로 이동
            outputs = model(images)   # 모델에서 예측 결과 얻기

            for image_id, output in zip(image_ids, outputs):   # 각 이미지에 대해 예측 결과를 반복
                for i in range(len(output["boxes"])):   # 예측된 바운딩 박스 수만큼 반복
                    box = output["boxes"][i].cpu().numpy()   # 바운딩 박스 좌표를 CPU로 이동 후 NumPy 배열로 변환
                    score = float(output["scores"][i].cpu().item())   # 예측 점수를 CPU로 이동 후 float 값으로 변환
                    category_id = int(output["labels"][i].cpu().item())   # 예측된 클래스 ID를 CPU로 이동 후 int 값으로 변환

                    # COCO 형식의 바운딩 박스 [x_min, y_min, width, height]로 변환
                    bbox = [float(box[0]), float(box[1]), float(box[2] - box[0]), float(box[3] - box[1])]

                    result = {   # 예측 결과를 딕셔너리로 저장
                        "image_id": image_id,
                        "category_id": category_id,
                        "bbox": bbox,
                        "score": score
                    }
                    results.append(result)   # 결과 리스트에 추가

    # 결과를 JSON 파일로 저장
    with open(output_file, "w") as f:   # 출력 파일 열기
        json.dump(results, f)   # JSON 형식으로 결과 저장

    print(f"Predictions saved to {output_file}")   # 저장 완료 메시지 출력

# 테스트 셋에 대해 예측을 수행하는 메인 함수
def main():
    # 데이터 로딩 코드
    test_dataset = CustomDataset('test', get_transform())   # 테스트 데이터셋 로드
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0, collate_fn=lambda x: tuple(zip(*x)))   # 테스트 데이터 로더 생성

    # 디바이스 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # 사용 가능한 디바이스 설정 (CUDA 또는 CPU)
    print(f"Using device: {device}")   # 디바이스 출력

    # 모델 설정
    num_classes = 11  # 10개의 클래스 + 배경 클래스 (background)
    model = get_model(num_classes, "model_1.pth")   # 모델 생성 및 저장된 가중치 로드
    model.to(device)   # 모델을 CUDA 디바이스로 이동

    # 예측 생성 및 저장
    save_predictions_as_coco_format(model, test_loader, device, output_file="predictions.json")   # 예측 생성 및 결과 저장

if __name__ == "__main__":   # 메인 프로그램 시작점
    main()   # 메인 함수 호출


In [ ]:
import os  # 운영체제와의 상호작용을 위한 os 모듈 임포트
import json  # JSON 파일을 처리하기 위한 json 모듈 임포트

# 입력 디렉토리의 바운딩 박스 데이터를 COCO 형식으로 변환하는 함수 정의
def convert_to_coco_format(input_dir, output_json):
    coco_format = {  # COCO 형식의 기본 구조를 정의
        "images": [],
        "annotations": [],
        "categories": []
    }

    # 카테고리 정의 (클래스 이름을 ID로 매핑)
    class_to_id = {  # 클래스 이름과 해당 클래스 ID를 매핑하는 딕셔너리 정의
        'car': 1, 'bus': 2, 'truck': 3, 'special vehicle': 4, 
        'motorcycle': 5, 'bicycle': 6, 'personal mobility': 7, 
        'person': 8, 'Traffic_light': 9, 'Traffic_sign': 10
    }

    # COCO 형식에 카테고리 추가
    for class_name, class_id in class_to_id.items():  # 각 클래스 이름과 ID에 대해 반복
        coco_format["categories"].append({  # 카테고리를 COCO 형식에 추가
            "id": class_id,  # 클래스 ID
            "name": class_name,  # 클래스 이름
            "supercategory": "none"  # 상위 카테고리 (없음)
        })

    # Annotation ID 초기화
    annotation_id = 1  # 어노테이션 ID를 1로 초기화

    # 각 레이블 파일 처리
    for label_file in os.listdir(input_dir):  # 입력 디렉토리 내의 파일을 반복
        if label_file.endswith('.json'):  # JSON 파일만 처리
            with open(os.path.join(input_dir, label_file), 'r') as f:  # JSON 파일 열기
                label_data = json.load(f)  # JSON 데이터를 파이썬 객체로 로드
            
            # 이미지 정보
            image_id = label_data["image_name"]  # 이미지 파일 이름을 ID로 사용
            width, height = label_data["image_size"]  # 이미지의 너비와 높이 정보 가져오기

            coco_format["images"].append({  # 이미지 정보를 COCO 형식에 추가
                "id": image_id,  # 이미지 ID
                "file_name": label_data["image_name"],  # 이미지 파일 이름
                "width": width,  # 이미지 너비
                "height": height  # 이미지 높이
            })

            # 어노테이션 처리
            for annotation in label_data["Annotation"]:  # 각 어노테이션을 반복
                class_name = annotation["class_name"]  # 어노테이션된 클래스 이름 가져오기
                bbox = annotation["data"]  # 바운딩 박스 좌표 가져오기
                x_min, y_min, x_max, y_max = bbox  # 바운딩 박스 좌표 분리
                width = x_max - x_min  # 바운딩 박스의 너비 계산
                height = y_max - y_min  # 바운딩 박스의 높이 계산

                coco_format["annotations"].append({  # 어노테이션을 COCO 형식에 추가
                    "id": annotation_id,  # 어노테이션 ID
                    "image_id": image_id,  # 어노테이션된 이미지 ID
                    "category_id": class_to_id[class_name],  # 어노테이션된 클래스 ID
                    "bbox": [x_min, y_min, width, height],  # 바운딩 박스 좌표
                    "area": width * height,  # 바운딩 박스 면적 계산
                    "iscrowd": 0  # 군중 어노테이션 여부 (아니오)
                })

                annotation_id += 1  # 어노테이션 ID 증가

    # COCO 형식 JSON 저장
    with open(output_json, 'w') as f:  # 출력 파일 열기
        json.dump(coco_format, f, indent=4)  # COCO 형식 데이터를 JSON 파일로 저장

    print(f"COCO format JSON saved to {output_json}")  # 저장 완료 메시지 출력

# 사용
input_dir = "test/labels"  # JSON 레이블 파일이 있는 입력 디렉토리 경로
output_json = "groundtruth_coco_format.json"  # 출력 COCO 형식 JSON 파일 경로
convert_to_coco_format(input_dir, output_json)  # 변환 함수 호출

In [ ]:
# 잘못 측정된 prediction 및 ground truth 체크

def check_image_id_compatibility(groundtruth_file, prediction_file):
    # Load ground truth and predictions
    with open(groundtruth_file, 'r') as f:
        groundtruth_data = json.load(f)
    
    with open(prediction_file, 'r') as f:
        predictions_data = json.load(f)

    # Extract image ids from ground truth
    groundtruth_image_ids = set(img['id'] for img in groundtruth_data['images'])
    
    # Extract image ids from predictions
    prediction_image_ids = set(pred['image_id'] for pred in predictions_data)
    
    # Check for mismatch
    if not groundtruth_image_ids.issubset(prediction_image_ids):
        print("Mismatch found between ground truth and predictions.")
        print("Missing in predictions:", groundtruth_image_ids - prediction_image_ids)
        print("Extra in predictions:", prediction_image_ids - groundtruth_image_ids)
    else:
        print("All image IDs match.")

# Example usage
check_image_id_compatibility('groundtruth_coco_format.json', 'predictions_corrected_0.json')

In [ ]:
# 잘못 측정된 prediction.json 및 groundtruth.json 수정

import json

def correct_image_id_format(prediction_file, output_file):
    # Load predictions
    with open(prediction_file, 'r') as f:
        predictions = json.load(f)
    
    # Add '.jpg' extension to each image_id
    for prediction in predictions:
        prediction['image_id'] = prediction['image_id'] + '.jpg'
    
    # Save corrected predictions
    with open(output_file, 'w') as f:
        json.dump(predictions, f, indent=4)

# Example usage
correct_image_id_format('predictions_0.json', 'predictions_corrected_0.json')

In [ ]:
import json  # JSON 파일을 처리하기 위한 json 모듈 임포트
import numpy as np  # 배열 연산을 위한 numpy 모듈 임포트
import torch  # PyTorch를 사용하기 위한 torch 모듈 임포트
from torchvision.ops import nms  # NMS(Non-Maximum Suppression) 함수를 사용하기 위해 임포트
from collections import defaultdict  # 기본값이 있는 딕셔너리를 생성하기 위한 defaultdict 임포트

# JSON 파일을 로드하는 함수 정의
def load_json(filename):
    with open(filename, 'r') as f:  # 파일 열기 (읽기 모드)
        return json.load(f)  # 파일 내용을 JSON 형식으로 읽어 반환

# JSON 파일을 저장하는 함수 정의
def save_json(data, filename):
    with open(filename, 'w') as f:  # 파일 열기 (쓰기 모드)
        json.dump(data, f, indent=4)  # 데이터를 JSON 형식으로 저장, 들여쓰기 설정

# 바운딩 박스를 [x, y, w, h] 형식에서 [x1, y1, x2, y2] 형식으로 변환하는 함수 정의
def to_xyxy(bbox):
    x, y, w, h = bbox  # 바운딩 박스 좌표를 각각 x, y, w, h로 분리
    return [x, y, x + w, y + h]  # x2와 y2 계산 후 [x1, y1, x2, y2]로 반환

# 메인 함수 정의, NMS를 적용하여 예측 결과를 필터링
def main(predictions_file, output_file, iou_threshold=0.5):
    # 예측 결과를 JSON 파일에서 로드
    data = load_json(predictions_file)
    
    # 이미지와 카테고리별로 탐지 결과를 그룹화
    grouped_detections = defaultdict(lambda: defaultdict(list))
    
    for item in data:  # JSON 데이터의 각 항목을 반복 처리
        image_id = item['image_id']  # 이미지 ID 추출
        category_id = item['category_id']  # 카테고리 ID 추출
        bbox = item['bbox']  # 바운딩 박스 정보 추출
        score = item['score']  # 신뢰도 점수 추출
        
        # 그룹화된 탐지 결과에 추가
        grouped_detections[image_id][category_id].append({
            'bbox': bbox,
            'score': score
        })
    
    # 각 카테고리와 이미지에 대해 NMS 적용
    filtered_detections = []  # 필터링된 탐지 결과를 저장할 리스트

    for image_id, categories in grouped_detections.items():  # 각 이미지에 대해 반복
        for category_id, items in categories.items():  # 각 카테고리에 대해 반복
            if len(items) == 0:  # 항목이 없으면 건너뜀
                continue

            bboxes = np.array([item['bbox'] for item in items])  # 바운딩 박스 리스트를 numpy 배열로 변환
            scores = np.array([item['score'] for item in items])  # 점수 리스트를 numpy 배열로 변환

            # 바운딩 박스를 [x, y, w, h]에서 [x1, y1, x2, y2] 형식으로 변환
            bboxes_xyxy = np.array([to_xyxy(bbox) for bbox in bboxes])

            # numpy 배열을 PyTorch 텐서로 변환
            bboxes_tensor = torch.tensor(bboxes_xyxy, dtype=torch.float32)
            scores_tensor = torch.tensor(scores, dtype=torch.float32)

            # NMS 수행
            keep_indices = nms(bboxes_tensor, scores_tensor, iou_threshold).numpy()

            # 필터링된 결과를 수집
            for idx in keep_indices:  # NMS 이후 남은 인덱스를 반복
                filtered_detections.append({
                    'image_id': image_id,  # 이미지 ID 추가
                    'category_id': category_id,  # 카테고리 ID 추가
                    'bbox': bboxes[idx].tolist(),  # 바운딩 박스 추가
                    'score': scores[idx].tolist()  # 점수 추가
                })
    
    # 필터링된 탐지 결과를 JSON 파일로 저장
    save_json(filtered_detections, output_file)

# 메인 함수 실행
if __name__ == "__main__":
    # 파일 경로 설정
    predictions_file = 'predictions_corrected_0.json'  # 입력 파일 경로
    output_file = 'predictions_nms_0.json'  # 출력 파일 경로
    
    main(predictions_file, output_file)  # 메인 함수 호출


In [ ]:
import json

# confidence threshold 제한
def filter_confidence_predictions(input_json, output_json, threshold=0.5):
    # predictions.json 파일을 엽니다.
    with open(input_json, 'r') as f:
        predictions = json.load(f)

    # score가 threshold보다 큰 예측만 남김
    filtered_predictions = [pred for pred in predictions if pred['score'] >= threshold]

    # 새로운 JSON 파일로 저장
    with open(output_json, 'w') as f:
        json.dump(filtered_predictions, f, indent=4)

# 사용 예시
input_json = 'predictions_nms_0.json'  # 기존 predictions.json 파일 경로
output_json = 'predictions_confidence_0.json'  # 저장할 파일 경로
threshold = 0.30  # score 임계값 설정

filter_confidence_predictions(input_json, output_json, threshold)


In [ ]:
from pycocotools.coco import COCO  # COCO 데이터셋을 처리하기 위한 COCO 클래스 임포트
from pycocotools.cocoeval import COCOeval  # COCO 데이터셋 평가를 위한 COCOeval 클래스 임포트
import json  # JSON 파일 처리를 위한 json 모듈 임포트

# mAP(Mean Average Precision) 계산 함수 정의
def calculate_map(groundtruth_file, prediction_file):
    # Load ground truth and predictions
    coco_gt = COCO(groundtruth_file)  # COCO 형식의 ground truth JSON 파일 로드
    coco_dt = coco_gt.loadRes(prediction_file)  # 예측 결과 JSON 파일 로드
    
    # Initialize COCOeval object
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')  # COCOeval 객체 초기화 (바운딩 박스 평가 모드)
    
    # coco_eval.params.maxDets = [1, 10, 40]  # 평가에서 고려할 최대 탐지 개수 설정 (옵션, 주석 처리됨)
    # coco_eval.params.iouThrs = [0.50, 0.75, 1.0]  # 평가에 사용할 IoU 임계값 설정 (옵션, 주석 처리됨)

    # Run the evaluation
    coco_eval.evaluate()  # 평가 실행 (이미지 단위로 일치 항목을 계산)
    coco_eval.accumulate()  # 평가 결과 누적
    coco_eval.summarize()  # 요약된 평가 결과 출력

    # Get the mAP score
    map_score = coco_eval.stats[0]  # mAP 점수 추출 (AP @[ IoU=0.50:0.95 | area=all | maxDets=100 ])
    print(f"mAP: {map_score:.4f}")  # mAP 점수 출력

# Example usage
groundtruth_file = 'groundtruth_coco_format.json'  # Ground truth JSON 파일 경로
prediction_file = 'predictions_nms_0.json'  # 예측 결과 JSON 파일 경로
calculate_map(groundtruth_file, prediction_file)  # mAP 계산 함수 호출


In [ ]:
import json  # JSON 파일을 처리하기 위한 모듈 임포트
import matplotlib.pyplot as plt  # 이미지 시각화를 위한 matplotlib 임포트 (사용되지 않음)
from PIL import Image, ImageDraw, ImageFont  # 이미지 처리를 위한 Pillow 모듈 임포트
from collections import defaultdict  # 기본값이 있는 딕셔너리 생성을 위한 defaultdict 임포트

# Ground truth 파일에서 카테고리 이름을 로드하는 함수 정의
def load_category_names(groundtruth_file):
    with open(groundtruth_file, 'r') as f:
        data = json.load(f)  # JSON 파일 로드
    categories = data.get('categories', [])  # 'categories' 키로부터 카테고리 정보 가져오기
    category_map = {category['id']: category['name'] for category in categories}  # 카테고리 ID를 이름으로 매핑
    return category_map  # 카테고리 맵 반환

# 이미지에 바운딩 박스 및 레이블을 그리는 함수 정의
def draw_bboxes(image_path, predictions, category_map, output_path):
    # Load image
    image = Image.open(image_path)  # 이미지를 열기
    draw = ImageDraw.Draw(image)  # 이미지를 그리기 위한 객체 생성
    
    # Optional: Load a font for the labels (requires a .ttf file)
    try:
        font = ImageFont.truetype("arial.ttf", 15)  # Arial 폰트 로드 (크기 15)
    except IOError:
        font = ImageFont.load_default()  # Arial 폰트가 없으면 기본 폰트 로드

    # 각 예측에 대해 바운딩 박스와 레이블 그리기
    for pred in predictions:
        bbox = pred['bbox']  # 바운딩 박스 좌표 가져오기
        category_id = pred['category_id']  # 카테고리 ID 가져오기
        score = pred['score']  # 예측 신뢰도 점수 가져오기
        
        # Convert [x_min, y_min, width, height] to [x0, y0, x1, y1]
        x_min, y_min, width, height = bbox  # 바운딩 박스 좌표 분리
        x0, y0 = x_min, y_min  # 좌상단 좌표
        x1, y1 = x_min + width, y_min + height  # 우하단 좌표
        
        # Draw the bounding box
        draw.rectangle([x0, y0, x1, y1], outline='red', width=2)  # 바운딩 박스 그리기
        
        # Get category name
        category_name = category_map.get(category_id, f'Unknown ({category_id})')  # 카테고리 이름 가져오기
        
        # Prepare the label
        label = f'{category_name}, {score:.2f}'  # 레이블 텍스트 생성
        
        # Draw the label
        draw.text((x0, y0 - 10), label, fill='red', font=font)  # 레이블 텍스트 그리기
    
    # Save the image with annotations
    image.save(output_path)  # 주석이 추가된 이미지를 저장

# 예측 파일을 처리하여 이미지를 그리기 위한 함수 정의
def process_predictions(predictions_file, images_folder, output_folder, groundtruth_file):
    # Load category names
    category_map = load_category_names(groundtruth_file)  # 카테고리 이름 로드
    
    # Load predictions
    with open(predictions_file, 'r') as f:
        predictions = json.load(f)  # 예측 JSON 파일 로드
    
    # Group predictions by image
    image_predictions = defaultdict(list)  # 이미지별로 예측을 그룹화할 defaultdict 생성
    for pred in predictions:
        image_id = pred['image_id']  # 이미지 ID 가져오기
        image_predictions[image_id].append(pred)  # 이미지 ID를 키로 하여 예측을 리스트에 추가
    
    # Process each image
    for image_id, preds in image_predictions.items():
        image_path = f'{images_folder}/{image_id}'  # 원본 이미지 경로 생성
        output_path = f'{output_folder}/{image_id}'  # 출력 이미지 경로 생성
        
        draw_bboxes(image_path, preds, category_map, output_path)  # 이미지에 바운딩 박스 및 레이블 그리기

# Example usage
predictions_file = 'predictions_nms_0.json'  # 예측 JSON 파일 경로
images_folder = 'test/images'  # 원본 이미지가 있는 폴더 경로
output_folder = 'test/images_nms_0'  # 출력 이미지가 저장될 폴더 경로
groundtruth_file = 'groundtruth_coco_format.json'  # Ground truth JSON 파일 경로

process_predictions(predictions_file, images_folder, output_folder, groundtruth_file)  # 예측 처리 함수 호출